In [1]:
# Utilidades del sistema
import os
import time

# Bibliotecas externas
from tqdm.auto import tqdm

# Procesamiento de documentos
from odf.opendocument import load
from odf.text import P

# LangChain y módulos relacionados
import langchain_community
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain.chains import RetrievalQA

# Pinecone y módulos relacionados
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore


# Obtiene las claves API necesarias desde las variables de entorno
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2") 
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT="pr-plaintive-radar-87"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # Obtiene la clave API de Pinecone desde las variables de entorno

assert OPENAI_API_KEY is not None, "OPENAI_API_KEY is not set"
assert LANGCHAIN_TRACING_V2 is not None, "LANGCHAIN_TRACING_V2 is not set"
assert LANGCHAIN_API_KEY is not None, "LANGCHAIN_API_KEY is not set"

# Inicializa el modelo de chat de OpenAI con configuración personalizada
chat = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,  # Utiliza la clave API para autenticar la conexión con OpenAI
    model='gpt-3.5-turbo',  # Especifica el modelo de lenguaje a usar
    temperature=0,  # Control de creatividad (0 hace que el output sea más determinista)
    streaming=True  # Permite la transmisión de resultados conforme se generan
)

In [2]:
### EMBEDDINGS
# Inicializa un modelo de embeddings de OpenAI con una versión específica
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")  # Crea un modelo de embeddings utilizando la versión "text-embedding-3-large"

# Imprime la longitud del vector generado por una consulta
print(len(embed_model.embed_query('hola')))  # Genera el embedding de la palabra 'hola' y muestra la longitud del vector


3072


In [3]:
# Ruta de la carpeta donde se encuentran los documentos .odt
carpeta_Belen = "docs_odt/Belen"
carpeta_Deshays = "docs_odt/Deshays"  

In [4]:
# Función para leer todos los documentos .odt en una carpeta
def leer_documentos_odt(carpeta):
    documentos = {}
    for archivo in os.listdir(carpeta):
        if archivo.endswith(".odt"):  # Verifica que el archivo sea .odt
            ruta_archivo = os.path.join(carpeta, archivo)
            # Cargar el archivo .odt
            documento = load(ruta_archivo)
            contenido = []
            # Extraer el texto de los párrafos
            for elemento in documento.getElementsByType(P):
                contenido.append(str(elemento))
            documentos[archivo] = "\n".join(contenido)
    return documentos

In [5]:
# Leemos los documentos en un diccionario {nombre_archivo: contenido}
docs_belen_dict = leer_documentos_odt(carpeta_Belen)
docs_deshays_dict = leer_documentos_odt(carpeta_Deshays)

In [6]:
docs_belen_dict

{'santiago_cv.odt': 'Santiago Francisco BELEN DEAS\nPersonal Information\nAddress: Av. Corrientes 16204piso Dto: “B” CP: 1042 Nationality: Argentine and Spanish.\nCity: Capital Federal, Buenos Aires, Argentina\nPhone Number: +54 9 11-5715-9118\nDate of Birth: 26/06/1984\nNational ID: 32.182.510\nE-mail: Sbelen@fmed.uba.ar\nProfessional Profile\nMedical Specialist in Internal Medicine and Clinical Research Professional with extensive knowledge in multiple therapeutic areas, currently pursuing a Specialization in Artificial Intelligence at the University of Buenos Aires.\nExperienced Teaching Physician in Clinical Examination and Internal Medicine, with a strong commitment to medical education and clinical supervision. Actively participated as a Sub-Investigator in Phase 2/3 clinical trials, contributing to pivotal studies, includ ing Pfizer’s RSV and Influenza vaccines (C3671013 / C4781004), Moderna’s RSV and Influenza vaccines (mRNA-1345-P301 / mRNA1010-P301), and Medicago’s COVID-19 v

In [7]:
# Ahora creamos la lista de Documents con su contenido real
documents_Belen = [Document(page_content=contenido, metadata={"source": nombre_archivo}) 
                   for nombre_archivo, contenido in docs_belen_dict.items()]

documents_Deshays = [Document(page_content=contenido, metadata={"source": nombre_archivo}) 
                     for nombre_archivo, contenido in docs_deshays_dict.items()]

In [8]:
documents_Belen

[Document(metadata={'source': 'santiago_cv.odt'}, page_content='Santiago Francisco BELEN DEAS\nPersonal Information\nAddress: Av. Corrientes 16204piso Dto: “B” CP: 1042 Nationality: Argentine and Spanish.\nCity: Capital Federal, Buenos Aires, Argentina\nPhone Number: +54 9 11-5715-9118\nDate of Birth: 26/06/1984\nNational ID: 32.182.510\nE-mail: Sbelen@fmed.uba.ar\nProfessional Profile\nMedical Specialist in Internal Medicine and Clinical Research Professional with extensive knowledge in multiple therapeutic areas, currently pursuing a Specialization in Artificial Intelligence at the University of Buenos Aires.\nExperienced Teaching Physician in Clinical Examination and Internal Medicine, with a strong commitment to medical education and clinical supervision. Actively participated as a Sub-Investigator in Phase 2/3 clinical trials, contributing to pivotal studies, includ ing Pfizer’s RSV and Influenza vaccines (C3671013 / C4781004), Moderna’s RSV and Influenza vaccines (mRNA-1345-P301 

In [9]:
# Mostramos los primeros 50 caracteres de cada documento de Belén
for idx, doc in enumerate(documents_Belen):
    print(f"Documento {idx + 1}:")
    print(f"Contenido (primeros 50 caracteres):\n{doc.page_content[:50]}")
    print("-" * 40)

Documento 1:
Contenido (primeros 50 caracteres):
Santiago Francisco BELEN DEAS
Personal Information
----------------------------------------


In [10]:
# Define una función para dividir los documentos en partes más pequeñas (chunks)
def chunk_data(docs, chunk_size=800, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)  # Inicializa el divisor de texto con el tamaño de chunk y solapamiento especificados
    doc = text_splitter.split_documents(docs)  # Divide los documentos en partes más pequeñas
    return doc  # Retorna los documentos divididos

# Ahora aplicamos la función chunk_data
Cv_Belen = chunk_data(docs=documents_Belen, chunk_size=3000, chunk_overlap=50)
Cv_Deshays = chunk_data(docs=documents_Deshays, chunk_size=3000, chunk_overlap=50)

In [11]:
Cv_Belen

[Document(metadata={'source': 'santiago_cv.odt'}, page_content='Santiago Francisco BELEN DEAS\nPersonal Information\nAddress: Av. Corrientes 16204piso Dto: “B” CP: 1042 Nationality: Argentine and Spanish.\nCity: Capital Federal, Buenos Aires, Argentina\nPhone Number: +54 9 11-5715-9118\nDate of Birth: 26/06/1984\nNational ID: 32.182.510\nE-mail: Sbelen@fmed.uba.ar\nProfessional Profile\nMedical Specialist in Internal Medicine and Clinical Research Professional with extensive knowledge in multiple therapeutic areas, currently pursuing a Specialization in Artificial Intelligence at the University of Buenos Aires.\nExperienced Teaching Physician in Clinical Examination and Internal Medicine, with a strong commitment to medical education and clinical supervision. Actively participated as a Sub-Investigator in Phase 2/3 clinical trials, contributing to pivotal studies, includ ing Pfizer’s RSV and Influenza vaccines (C3671013 / C4781004), Moderna’s RSV and Influenza vaccines (mRNA-1345-P301 

In [12]:
# Revisamos que ahora Cv_Belen contenga los chunks correctamente
for chunk in Cv_Belen[:2]:
    print(chunk.page_content[:100])
    print("----")

Santiago Francisco BELEN DEAS
Personal Information
Address: Av. Corrientes 16204piso Dto: “B” CP: 10
----
2015 – 2018
Teaching Physician in Clinical Examination and Internal Medicine 5th Chair of Internal M
----


---


In [13]:
# CARGA LOS DOCUMENTOS Y VECTORES A LA BASE DE DATOS PINECONE
## CONEXIÓN CON LA BASE DE DATOS DE PINECONE
from pinecone import Pinecone, ServerlessSpec


# Conecta a la base de datos Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)  # Inicializa la conexión a Pinecone utilizando la clave API
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'  # Obtiene la configuración de la nube, por defecto 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'  # Obtiene la región de Pinecone, por defecto 'us-east-1'
spec = ServerlessSpec(cloud=cloud, region=region)  # Crea una especificación sin servidor para la conexión a Pinecone
index_name = 'llm-tp2'  # Nombre del índice en Pinecone

# Elimina el índice si ya existe en Pinecone
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)  # Elimina el índice si ya existe
    print("index {} borrado".format(index_name))  # Imprime un mensaje indicando que el índice fue borrado

# Verifica si el índice no existe y lo crea
if index_name not in pc.list_indexes().names():
    print("index creado con el nombre: {}".format(index_name))  # Imprime un mensaje indicando que el índice será creado
    pc.create_index(
        index_name,
        dimension=3072,  # Dimensionalidad del modelo de embedding
        metric='cosine',  # Métrica para comparar vectores (usando distancia coseno)
        spec=spec  # Especificación de conexión
    )
else:
    print("el index con el nombre {} ya estaba creado".format(index_name))  # Imprime un mensaje si el índice ya existía


index llm-tp2 borrado
index creado con el nombre: llm-tp2


---

In [14]:
## INSERCIÓN DE LOS VECTORES EN LA BASE DE DATOS DE PINECONE

# Define diferentes espacios de nombres para tesis y currículum
namespace_Belen = "espacio_Belen"  # Espacio de nombres para el CV Belen
namespace_Deshays = "espacio_Deshays"  # Espacio de nombres para el CV Deshays

# Inserta los documentos en la base de datos Pinecone
docsearch_Belen = PineconeVectorStore.from_documents(
    documents=Cv_Belen,  # Documentos de tesis divididos a insertar
    index_name=index_name,  # Nombre del índice en Pinecone
    embedding=embed_model,  # Modelo de embeddings a usar
    namespace=namespace_Belen  # Espacio de nombres definido para tesis
)

print("upserted values to {} index under namespace {}".format(index_name, namespace_Belen))  # Imprime un mensaje indicando que los valores fueron insertados

# Inserta los documentos en la base de datos Pinecone
docsearch_Deshays = PineconeVectorStore.from_documents(
    documents=Cv_Deshays,  # Documentos de currículum divididos a insertar
    index_name=index_name,  # Nombre del índice en Pinecone
    embedding=embed_model,  # Modelo de embeddings a usar
    namespace=namespace_Deshays  # Espacio de nombres definido para currículum
)
print("upserted values to {} index under namespace {}".format(index_name, namespace_Deshays))  # Imprime un mensaje indicando que los valores fueron insertados

time.sleep(1)  # Pausa de 1 segundo


upserted values to llm-tp2 index under namespace espacio_Belen
upserted values to llm-tp2 index under namespace espacio_Deshays


---


In [15]:
# Reconectar a Pinecone
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")  # Obtiene la clave API de Pinecone desde las variables de entorno nuevamente
pc = Pinecone(api_key=PINECONE_API_KEY)  # Inicializa la conexión a Pinecone utilizando la clave API
index_name = 'llm-tp2'  # Nombre del índice principal

# Inicializa el vectorstore para los documentos de Belen
vectorstore_Belen = PineconeVectorStore(
    index_name=index_name,  # Nombre del índice
    embedding=embed_model,  # Modelo de embeddings a usar
    namespace=namespace_Belen  # Espacio de nombres definido para tesis
)
retriever_Belen = vectorstore_Belen.as_retriever()  # Crea un objeto para recuperar documentos del índice de tesis

# Inicializa el vectorstore para los documentos de currículum
vectorstore_Deshays = PineconeVectorStore(
    index_name=index_name,  # Nombre del índice
    embedding=embed_model,  # Modelo de embeddings a usar
    namespace=namespace_Deshays  # Espacio de nombres definido para currículum
)
retriever_Deshays = vectorstore_Deshays.as_retriever()  # Crea un objeto para recuperar documentos del índice de currículum


---


In [19]:

# Realiza una búsqueda de similitud en el vectorstore de tesis
query = "in which companies did santiago used to work"  # Define la consulta para realizar la búsqueda
documents_result = vectorstore_Belen.similarity_search(query, k=1)  # Realiza la búsqueda de los documentos más similares con k=1

documents_result


[Document(id='d102115d-4354-4fc1-8d59-5363e94e0297', metadata={'source': 'santiago_cv.odt'}, page_content='Santiago Francisco BELEN DEAS\nPersonal Information\nAddress: Av. Corrientes 16204piso Dto: “B” CP: 1042 Nationality: Argentine and Spanish.\nCity: Capital Federal, Buenos Aires, Argentina\nPhone Number: +54 9 11-5715-9118\nDate of Birth: 26/06/1984\nNational ID: 32.182.510\nE-mail: Sbelen@fmed.uba.ar\nProfessional Profile\nMedical Specialist in Internal Medicine and Clinical Research Professional with extensive knowledge in multiple therapeutic areas, currently pursuing a Specialization in Artificial Intelligence at the University of Buenos Aires.\nExperienced Teaching Physician in Clinical Examination and Internal Medicine, with a strong commitment to medical education and clinical supervision. Actively participated as a Sub-Investigator in Phase 2/3 clinical trials, contributing to pivotal studies, includ ing Pfizer’s RSV and Influenza vaccines (C3671013 / C4781004), Moderna’s 

---

In [20]:
# Realiza una consulta para obtener información sobre experiencia laboral en los documentos de tesis
document_query_Belen = "does Santiago have any haircut experience"  # Define la consulta para obtener información específica sobre la tesis

# Inicializa la cadena de recuperación y respuesta para los documentos de tesis
qa_Belen = RetrievalQA.from_chain_type(  
    llm=chat,  # Utiliza el modelo de lenguaje configurado previamente
    chain_type="stuff",  # Tipo de cadena utilizada para unir las respuestas
    retriever=vectorstore_Belen.as_retriever()  # Utiliza el objeto retriever para obtener los documentos relevantes
)  

# Realiza la consulta en los documentos de tesis
result_Belen = qa_Belen.invoke(document_query_Belen)  # Invoca la consulta para recuperar información de la tesis

# Imprime el resultado de la consulta en los documentos de tesis
print("Resultado de la consulta en documentos Belen:")
print(result_Belen['result'])  # Imprime la parte 'result' del resultado
print(result_Belen)  # Imprime el resultado completo

# Realiza una consulta para obtener información sobre experiencia laboral en los documentos de currículum
document_query_Deshays = "does octavio have any neural network experience"  # Define la consulta para obtener información específica sobre el currículum

# Inicializa la cadena de recuperación y respuesta para los documentos de currículum
qa_Deshays = RetrievalQA.from_chain_type(  
    llm=chat,  # Utiliza el modelo de lenguaje configurado previamente
    chain_type="stuff",  # Tipo de cadena utilizada para unir las respuestas
    retriever=vectorstore_Deshays.as_retriever()  # Utiliza el objeto retriever para obtener los documentos relevantes del índice de currículum
) 

# Realiza la consulta en el currículum
result_Deshay = qa_Deshays.invoke(document_query_Deshays)  # Invoca la consulta para recuperar información del currículum

# Imprime el resultado de la consulta en los documentos de currículum
print("Resultado de la consulta en documentos Deshay:")
print(result_Deshay['result'])  # Imprime la parte 'result' del resultado
print(result_Deshay)  # Imprime el resultado completo

Resultado de la consulta en documentos Belen:
Based on the provided information, there is no mention of Santiago Francisco BELEN DEAS having any experience or qualifications related to haircuts or hairstyling.
{'query': 'does Santiago have any haircut experience', 'result': 'Based on the provided information, there is no mention of Santiago Francisco BELEN DEAS having any experience or qualifications related to haircuts or hairstyling.'}
Resultado de la consulta en documentos Deshay:
Yes, Octavio Deshays has experience with neural networks. He has worked with Multimodal Neural Networks, CNN, and CLIP based classifiers in his projects as a Machine Learning Engineer.
{'query': 'does octavio have any neural network experience', 'result': 'Yes, Octavio Deshays has experience with neural networks. He has worked with Multimodal Neural Networks, CNN, and CLIP based classifiers in his projects as a Machine Learning Engineer.'}
